In [318]:
import random

In [319]:
def generate_plinko_grid(width, height):
    grid = {}
    for y in range(height):
        for x in range(width):
            if (y % 2 == 0 and x % 2 == 1) or (y % 2 == 1 and x % 2 == 0):
                grid[(x, height - 1 - y)] = 'O'  # place pegs in a checkered pattern
            else:
                grid[(x, height - 1 - y)] = ' '  # empty spaces between pegs
    return grid

def mark_ledge(grid, start_x, length, ledge_y, ledges):
    if ledge_y not in ledges:
        ledges[ledge_y] = 0  # initialize ledge visit counter for the row
    for x in range(start_x, start_x + length):
        grid[(x, ledge_y)] = '_'  # mark ledge locations

def mark_slide(grid, start_x, start_y, length, direction):
    slide_char = '\\' if direction == "forward" else '/'
    x, y = start_x, start_y
    
    for _ in range(length):
        if (x, y) in grid and grid[(x, y)] == 'O':
            grid[(x, y)] = slide_char  # replace pegs with slides
        
        # replace diagonally in the selected direction
        if direction == "forward":
            x += 1
            y -= 1
        else:
            x -= 1
            y -= 1

def mark_buckets(width, num_buckets):
    buckets = {}
    base_size = width // num_buckets
    extra = width % num_buckets  # distribute leftover columns
    middle_bucket = num_buckets // 2  # make the middle bucket smaller if needed
    start_x = 0
    
    for i in range(num_buckets):
        size = base_size + (1 if extra > 0 and i != middle_bucket else 0)
        for x in range(start_x, start_x + size):
            buckets[x] = i  # assign bucket numbers
        start_x += size
        if extra > 0 and i != middle_bucket:
            extra -= 1
    
    return buckets

def visualize_grid(grid, width, height, ball_position=None, buckets=None):
    print("   " + " ".join(str(i % 10) for i in range(width)))  # column labels
    for y in range(height - 1, -1, -1):
        row = f"{y:2} "  # row labels
        for x in range(width):
            if ball_position and (x, y) == ball_position:
                row += 'X'  # show ball position
            else:
                row += grid.get((x, y), ' ')
            row += " "
        print(row)
    print("=" * (2 * width))  # visual separator
    
    # print buckets below the board
    bucket_row = "   "
    for x in range(width):
        bucket_row += str(buckets.get(x, ' ')) + " " if buckets else "  "
    print(bucket_row)

def drop_ball(grid, width, height, start_x, ledges, buckets):
    x, y = start_x, height - 1
    while y > 0:
        visualize_grid(grid, width, height, (x, y), buckets)

        # check if the ball is on a slide
        while (x, y) in grid and grid[(x, y)] in {'\\', '/'}:
            if grid[(x, y)] == '\\':
                x += 1  # move right
            elif grid[(x, y)] == '/':
                x -= 1  # move left
            y -= 1
            if (x, y) not in grid or grid[(x, y)] == 'O':
                break  # stop sliding at next peg

        # check if the ball is on a ledge
        if y in ledges and grid.get((x, y)) == '_':
            ledges[y] += 1  # track visits to ledges by row
            ledge_positions = [col for col in range(width) if grid.get((col, y)) == '_']

            if ledge_positions:
                x = random.choice(ledge_positions)  # pick a new column
                print(f"Ball hit a ledge at row {y}. Moving to column {x}.")
                
                # ball falls straight down after ledge selection
                while (x, y - 1) in grid and grid[(x, y - 1)] == ' ':
                    y -= 1  # keep falling through open spaces
                y -= 1

        # determine possible diagonal movement
        possible_moves = []
        if (x - 1, y - 1) in grid and grid[(x - 1, y - 1)] in {'O', '_', '\\', '/'}:
            possible_moves.append((x - 1, y - 1))
        if (x + 1, y - 1) in grid and grid[(x + 1, y - 1)] in {'O', '_', '\\', '/'}:
            possible_moves.append((x + 1, y - 1))

        if possible_moves:
            x, y = random.choice(possible_moves)  # randomly choose diagonal direction
        else:
            y -= 1  # fall straight down if no diagonal move available

    bucket = buckets.get(x, 'Unknown') # determine bucket based on final x coordinate
    print(f"Ball landed in bucket {bucket} at column {x}.")
    visualize_grid(grid, width, height, (x, y), buckets)
    
    return bucket

In [320]:
width, height = 11, 10
num_buckets = 3
plinko_grid = generate_plinko_grid(width, height)

ledges = {}  # dictionary to track ledges and visits
mark_ledge(plinko_grid, start_x=4, length=3, ledge_y=8, ledges=ledges)
mark_ledge(plinko_grid, start_x=2, length=3, ledge_y=6, ledges=ledges)
mark_ledge(plinko_grid, start_x=6, length=2, ledge_y=4, ledges=ledges)
mark_ledge(plinko_grid, start_x=1, length=4, ledge_y=2, ledges=ledges)
mark_ledge(plinko_grid, start_x=5, length=3, ledge_y=1, ledges=ledges)

mark_slide(plinko_grid, start_x=0, start_y=6, length=3, direction="forward")
mark_slide(plinko_grid, start_x=10, start_y=8, length=3, direction="backward")

buckets = mark_buckets(width, num_buckets) # map each x value to a bucket

bucket_counts = {i: 0 for i in range(num_buckets)}
for _ in range(100):
    bucket = drop_ball(plinko_grid, width, height, start_x=random.randint(0, width - 1), ledges=ledges, buckets=buckets)
    bucket_counts[bucket] += 1

print("\nFinal Bucket Statistics:")
for bucket, count in sorted(bucket_counts.items()):
    print(f"Bucket {bucket}: {count} balls")

print("\nLedge Visit Statistics:")
for ledge in sorted(ledges.keys(), reverse=True):
    visits = ledges[ledge]
    print(f"Ledge at row {ledge}: {visits} visits")


   0 1 2 3 4 5 6 7 8 9 0
 9   O   O   O   O   X   
 8 O   O   _ _ _   O   / 
 7   O   O   O   O   /   
 6 \   _ _ _   O   /   O 
 5   \   O   O   O   O   
 4 O   \   O   _ _ O   O 
 3   O   O   O   O   O   
 2 O _ _ _ _   O   O   O 
 1   O   O   _ _ _   O   
 0 O   O   O   O   O   O 
   0 0 0 0 1 1 1 2 2 2 2 
   0 1 2 3 4 5 6 7 8 9 0
 9   O   O   O   O   O   
 8 O   O   _ _ _   O   X 
 7   O   O   O   O   /   
 6 \   _ _ _   O   /   O 
 5   \   O   O   O   O   
 4 O   \   O   _ _ O   O 
 3   O   O   O   O   O   
 2 O _ _ _ _   O   O   O 
 1   O   O   _ _ _   O   
 0 O   O   O   O   O   O 
   0 0 0 0 1 1 1 2 2 2 2 
   0 1 2 3 4 5 6 7 8 9 0
 9   O   O   O   O   O   
 8 O   O   _ _ _   O   / 
 7   O   O   O   O   /   
 6 \   _ _ _   O   /   O 
 5   \   O   O   O   O   
 4 O   \   O   X _ O   O 
 3   O   O   O   O   O   
 2 O _ _ _ _   O   O   O 
 1   O   O   _ _ _   O   
 0 O   O   O   O   O   O 
   0 0 0 0 1 1 1 2 2 2 2 
Ball hit a ledge at row 4. Moving to column 6.
   0 1 2 3 4 5 6 7 8